In [1]:
from src.SB.soi import SOI
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from pytorch_lightning.loggers import TensorBoardLogger
from src.benchmark.toy_dataset import Task_redundant ,Task_synergy,Task_combination
from pytorch_lightning.trainer import seed_everything
seed_everything(33)


Seed set to 33


33

In [2]:
nb_mod = 6
dim=5
r = {} 

def get_samples(test_loader,mod_list,device):
        data ={ mod :torch.Tensor().to(device) for mod in mod_list} 
        for batch in test_loader:
            for mod in mod_list:
                data[mod]  = torch.cat([data[mod],batch[mod].to(device) ] )
        return data
    

def test_sgima(rho):
    task = Task_combination(tasks= [Task_redundant(nb_var=3, rho=rho ,dim = dim),
                                    Task_redundant(nb_var=3, rho=rho,dim = dim),
                                
                               ]  ,dim=dim)
    N =  1000 *100
    nb_mod = task.nb_var
    d_train, d_test = task.get_torch_dataset(N,10*1000,dim=dim,rescale=False)

    train_loader = DataLoader(d_train, batch_size=256,shuffle=True,
                                num_workers=8, drop_last=True)

    test_loader = DataLoader(d_test, batch_size=1000,
                                shuffle= False,
                                num_workers=8, drop_last=False)

    mod_list={ "x"+ str(i) : dim for i in range(nb_mod) }

    test_samples = get_samples(test_loader, mod_list=mod_list,device = "cuda")

    model = SOI(
                    test_samples= test_samples,
                    gt = task.get_summary(),
                    lr=1e-2,
                    mod_list =mod_list ,
                    use_ema= True, 
                    debias=True,
                    test_epoch= 10,
                    tx=True,
                    scores_order= 1,
                    debug= True,
                    margin_time=1,
                    fill_zeros= True,
                    weight_subsets=True )

    CHECKPOINT_DIR = "trained_models/"
    tb_logger =  TensorBoardLogger(save_dir = CHECKPOINT_DIR,name="test"+str(dim))
    trainer = pl.Trainer( logger= tb_logger,
                        accelerator='gpu', devices= 1,
                            max_epochs= 50, 
                            default_root_dir = CHECKPOINT_DIR,
                        )

    trainer.fit(model=model, train_dataloaders=train_loader,val_dataloaders=test_loader  )
    return {"gt": task.get_summary(), "soi": model.compute_o_inf(test_samples, debias=True,nb_iter= 10)}
    


In [3]:
out = test_sgima(0.5)

starting to sample data
after cov
after reshape


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Weighting the scores to learn 



  | Name      | Type | Params
-----------------------------------
0 | score     | DiT  | 658 K 
1 | model_ema | EMA  | 658 K 
-----------------------------------
1.3 M     Trainable params
0         Non-trainable params
1.3 M     Total params
5.265     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
out["gt"]["o_inf"]

In [ ]:
out["soi"]["simple"]["o_inf"]